# Day 3: Delta Hedging

## Learning Objectives
- Understand delta hedging theory and Greeks
- Implement dynamic hedging strategies
- Analyze transaction costs impact on hedging
- Perform P&L analysis of hedged positions

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from dataclasses import dataclass
from typing import Tuple, List, Optional
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

## 1. Delta Hedging Theory

### The Greeks

**Delta (Δ)**: Sensitivity of option price to underlying price
$$\Delta = \frac{\partial V}{\partial S}$$

For a European call option under Black-Scholes:
$$\Delta_{call} = N(d_1)$$

For a European put option:
$$\Delta_{put} = N(d_1) - 1$$

Where:
$$d_1 = \frac{\ln(S/K) + (r + \sigma^2/2)T}{\sigma\sqrt{T}}$$

### Delta Hedging Principle

A delta-neutral portfolio has zero sensitivity to small changes in the underlying:
$$\Pi = V - \Delta \cdot S$$

The hedge ratio tells us how many shares to hold to neutralize the option position.

In [ ]:
@dataclass
class OptionParams:
    """Option parameters container."""
    S: float      # Spot price
    K: float      # Strike price
    T: float      # Time to maturity (years)
    r: float      # Risk-free rate
    sigma: float  # Volatility
    

class BlackScholesModel:
    """Black-Scholes option pricing and Greeks."""
    
    @staticmethod
    def d1(S: float, K: float, T: float, r: float, sigma: float) -> float:
        """Calculate d1 parameter."""
        if T <= 0:
            return np.inf if S > K else -np.inf
        return (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    
    @staticmethod
    def d2(S: float, K: float, T: float, r: float, sigma: float) -> float:
        """Calculate d2 parameter."""
        if T <= 0:
            return np.inf if S > K else -np.inf
        return BlackScholesModel.d1(S, K, T, r, sigma) - sigma * np.sqrt(T)
    
    @staticmethod
    def call_price(S: float, K: float, T: float, r: float, sigma: float) -> float:
        """Calculate call option price."""
        if T <= 0:
            return max(S - K, 0)
        d1 = BlackScholesModel.d1(S, K, T, r, sigma)
        d2 = BlackScholesModel.d2(S, K, T, r, sigma)
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    
    @staticmethod
    def put_price(S: float, K: float, T: float, r: float, sigma: float) -> float:
        """Calculate put option price."""
        if T <= 0:
            return max(K - S, 0)
        d1 = BlackScholesModel.d1(S, K, T, r, sigma)
        d2 = BlackScholesModel.d2(S, K, T, r, sigma)
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    @staticmethod
    def delta_call(S: float, K: float, T: float, r: float, sigma: float) -> float:
        """Calculate call option delta."""
        if T <= 0:
            return 1.0 if S > K else 0.0
        d1 = BlackScholesModel.d1(S, K, T, r, sigma)
        return norm.cdf(d1)
    
    @staticmethod
    def delta_put(S: float, K: float, T: float, r: float, sigma: float) -> float:
        """Calculate put option delta."""
        if T <= 0:
            return -1.0 if S < K else 0.0
        return BlackScholesModel.delta_call(S, K, T, r, sigma) - 1
    
    @staticmethod
    def gamma(S: float, K: float, T: float, r: float, sigma: float) -> float:
        """Calculate option gamma (same for call and put)."""
        if T <= 0:
            return 0.0
        d1 = BlackScholesModel.d1(S, K, T, r, sigma)
        return norm.pdf(d1) / (S * sigma * np.sqrt(T))
    
    @staticmethod
    def theta_call(S: float, K: float, T: float, r: float, sigma: float) -> float:
        """Calculate call option theta (per year)."""
        if T <= 0:
            return 0.0
        d1 = BlackScholesModel.d1(S, K, T, r, sigma)
        d2 = BlackScholesModel.d2(S, K, T, r, sigma)
        term1 = -S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
        term2 = -r * K * np.exp(-r * T) * norm.cdf(d2)
        return term1 + term2
    
    @staticmethod
    def vega(S: float, K: float, T: float, r: float, sigma: float) -> float:
        """Calculate option vega (same for call and put)."""
        if T <= 0:
            return 0.0
        d1 = BlackScholesModel.d1(S, K, T, r, sigma)
        return S * np.sqrt(T) * norm.pdf(d1)


# Test the model
bs = BlackScholesModel()
params = OptionParams(S=100, K=100, T=0.25, r=0.05, sigma=0.2)

print("Black-Scholes Model Test")
print("=" * 40)
print(f"Spot: ${params.S}, Strike: ${params.K}")
print(f"Time to Maturity: {params.T*252:.0f} days")
print(f"Risk-free Rate: {params.r*100:.1f}%")
print(f"Volatility: {params.sigma*100:.1f}%")
print()
print(f"Call Price: ${bs.call_price(params.S, params.K, params.T, params.r, params.sigma):.4f}")
print(f"Put Price: ${bs.put_price(params.S, params.K, params.T, params.r, params.sigma):.4f}")
print(f"Call Delta: {bs.delta_call(params.S, params.K, params.T, params.r, params.sigma):.4f}")
print(f"Gamma: {bs.gamma(params.S, params.K, params.T, params.r, params.sigma):.4f}")
print(f"Vega: {bs.vega(params.S, params.K, params.T, params.r, params.sigma):.4f}")
print(f"Theta (call, per day): {bs.theta_call(params.S, params.K, params.T, params.r, params.sigma)/252:.4f}")

### Delta Surface Visualization

In [ ]:
# Visualize delta across spot prices and time to maturity
spot_range = np.linspace(80, 120, 100)
time_range = np.array([0.01, 0.1, 0.25, 0.5, 1.0])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Delta vs Spot for different times
ax1 = axes[0]
for T in time_range:
    deltas = [bs.delta_call(S, 100, T, 0.05, 0.2) for S in spot_range]
    ax1.plot(spot_range, deltas, label=f'T = {T:.2f}y')

ax1.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
ax1.axvline(x=100, color='gray', linestyle='--', alpha=0.5)
ax1.set_xlabel('Spot Price ($)')
ax1.set_ylabel('Delta')
ax1.set_title('Call Delta vs Spot Price')
ax1.legend()
ax1.set_ylim(-0.05, 1.05)

# Gamma vs Spot for different times
ax2 = axes[1]
for T in time_range:
    gammas = [bs.gamma(S, 100, T, 0.05, 0.2) for S in spot_range]
    ax2.plot(spot_range, gammas, label=f'T = {T:.2f}y')

ax2.axvline(x=100, color='gray', linestyle='--', alpha=0.5)
ax2.set_xlabel('Spot Price ($)')
ax2.set_ylabel('Gamma')
ax2.set_title('Gamma vs Spot Price')
ax2.legend()

plt.tight_layout()
plt.show()

print("Key Observations:")
print("- Delta approaches step function as expiration nears")
print("- Gamma peaks at-the-money and increases near expiration")
print("- Higher gamma = more frequent rebalancing needed")

## 2. Dynamic Hedging Implementation

### Stock Price Simulation (Geometric Brownian Motion)

$$dS_t = \mu S_t dt + \sigma S_t dW_t$$

Discretized:
$$S_{t+\Delta t} = S_t \exp\left((\mu - \frac{\sigma^2}{2})\Delta t + \sigma\sqrt{\Delta t}Z\right)$$

In [ ]:
class GBMSimulator:
    """Geometric Brownian Motion simulator for stock prices."""
    
    def __init__(self, S0: float, mu: float, sigma: float, seed: int = None):
        self.S0 = S0
        self.mu = mu
        self.sigma = sigma
        if seed is not None:
            np.random.seed(seed)
    
    def simulate_path(self, T: float, n_steps: int) -> Tuple[np.ndarray, np.ndarray]:
        """Simulate a single price path."""
        dt = T / n_steps
        times = np.linspace(0, T, n_steps + 1)
        
        # Generate random increments
        Z = np.random.standard_normal(n_steps)
        
        # Calculate log returns
        log_returns = (self.mu - 0.5 * self.sigma**2) * dt + self.sigma * np.sqrt(dt) * Z
        
        # Cumulative sum for log prices
        log_prices = np.zeros(n_steps + 1)
        log_prices[0] = np.log(self.S0)
        log_prices[1:] = np.log(self.S0) + np.cumsum(log_returns)
        
        prices = np.exp(log_prices)
        return times, prices
    
    def simulate_paths(self, T: float, n_steps: int, n_paths: int) -> Tuple[np.ndarray, np.ndarray]:
        """Simulate multiple price paths."""
        dt = T / n_steps
        times = np.linspace(0, T, n_steps + 1)
        
        # Generate random increments for all paths
        Z = np.random.standard_normal((n_paths, n_steps))
        
        # Calculate log returns
        log_returns = (self.mu - 0.5 * self.sigma**2) * dt + self.sigma * np.sqrt(dt) * Z
        
        # Cumulative sum for log prices
        log_prices = np.zeros((n_paths, n_steps + 1))
        log_prices[:, 0] = np.log(self.S0)
        log_prices[:, 1:] = np.log(self.S0) + np.cumsum(log_returns, axis=1)
        
        prices = np.exp(log_prices)
        return times, prices


# Simulate sample paths
simulator = GBMSimulator(S0=100, mu=0.05, sigma=0.2, seed=42)
times, paths = simulator.simulate_paths(T=0.25, n_steps=63, n_paths=10)

plt.figure(figsize=(12, 5))
for i in range(10):
    plt.plot(times * 252, paths[i], alpha=0.7)
plt.xlabel('Trading Days')
plt.ylabel('Stock Price ($)')
plt.title('Simulated Stock Price Paths (GBM)')
plt.axhline(y=100, color='black', linestyle='--', label='Initial Price')
plt.legend()
plt.show()

### Delta Hedging Simulator

In [ ]:
@dataclass
class HedgingResult:
    """Container for hedging simulation results."""
    times: np.ndarray
    prices: np.ndarray
    deltas: np.ndarray
    option_values: np.ndarray
    stock_positions: np.ndarray
    cash_positions: np.ndarray
    portfolio_values: np.ndarray
    hedge_errors: np.ndarray
    transaction_costs: np.ndarray
    cumulative_costs: np.ndarray
    pnl: np.ndarray


class DeltaHedger:
    """Delta hedging simulator for options."""
    
    def __init__(self, K: float, T: float, r: float, sigma: float, 
                 is_call: bool = True, position: str = 'short'):
        """
        Initialize delta hedger.
        
        Args:
            K: Strike price
            T: Time to maturity
            r: Risk-free rate
            sigma: Volatility
            is_call: True for call option, False for put
            position: 'short' or 'long' option position
        """
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.is_call = is_call
        self.position = position  # 'short' means we sold the option
        self.bs = BlackScholesModel()
    
    def get_delta(self, S: float, tau: float) -> float:
        """Get delta for current market conditions."""
        if self.is_call:
            delta = self.bs.delta_call(S, self.K, tau, self.r, self.sigma)
        else:
            delta = self.bs.delta_put(S, self.K, tau, self.r, self.sigma)
        
        # If short option, we need opposite position in stock
        if self.position == 'short':
            return delta  # Buy delta shares to hedge short call
        else:
            return -delta  # Sell delta shares to hedge long call
    
    def get_option_value(self, S: float, tau: float) -> float:
        """Get option value for current market conditions."""
        if self.is_call:
            return self.bs.call_price(S, self.K, tau, self.r, self.sigma)
        else:
            return self.bs.put_price(S, self.K, tau, self.r, self.sigma)
    
    def simulate_hedging(self, prices: np.ndarray, times: np.ndarray,
                         rebalance_freq: int = 1,
                         transaction_cost_rate: float = 0.0) -> HedgingResult:
        """
        Simulate delta hedging along a price path.
        
        Args:
            prices: Array of stock prices
            times: Array of time points
            rebalance_freq: Rebalance every n steps
            transaction_cost_rate: Proportional transaction cost
        """
        n_steps = len(prices)
        
        # Initialize arrays
        deltas = np.zeros(n_steps)
        option_values = np.zeros(n_steps)
        stock_positions = np.zeros(n_steps)  # Number of shares
        cash_positions = np.zeros(n_steps)   # Cash in bank account
        portfolio_values = np.zeros(n_steps)
        transaction_costs = np.zeros(n_steps)
        cumulative_costs = np.zeros(n_steps)
        
        # Initial setup
        S0 = prices[0]
        tau0 = self.T - times[0]
        
        option_values[0] = self.get_option_value(S0, tau0)
        deltas[0] = self.get_delta(S0, tau0)
        
        # Initial hedge: buy delta shares
        stock_positions[0] = deltas[0]
        
        # Initial cash: received option premium, paid for stock
        if self.position == 'short':
            # Sold option, received premium, bought stock
            cash_positions[0] = option_values[0] - stock_positions[0] * S0
        else:
            # Bought option, paid premium, shorted stock
            cash_positions[0] = -option_values[0] - stock_positions[0] * S0
        
        transaction_costs[0] = abs(stock_positions[0] * S0) * transaction_cost_rate
        cash_positions[0] -= transaction_costs[0]
        cumulative_costs[0] = transaction_costs[0]
        
        portfolio_values[0] = stock_positions[0] * S0 + cash_positions[0]
        
        # Simulate through time
        for i in range(1, n_steps):
            S = prices[i]
            tau = max(self.T - times[i], 0)
            dt = times[i] - times[i-1]
            
            # Update option value
            option_values[i] = self.get_option_value(S, tau)
            
            # Cash earns risk-free rate
            cash_positions[i] = cash_positions[i-1] * np.exp(self.r * dt)
            
            # Check if we should rebalance
            if i % rebalance_freq == 0:
                # Calculate new delta
                deltas[i] = self.get_delta(S, tau)
                
                # Calculate shares to trade
                shares_to_trade = deltas[i] - stock_positions[i-1]
                
                # Update positions
                stock_positions[i] = deltas[i]
                
                # Pay for new shares (or receive cash from selling)
                cash_positions[i] -= shares_to_trade * S
                
                # Transaction costs
                transaction_costs[i] = abs(shares_to_trade * S) * transaction_cost_rate
                cash_positions[i] -= transaction_costs[i]
            else:
                # No rebalancing
                deltas[i] = deltas[i-1]
                stock_positions[i] = stock_positions[i-1]
            
            cumulative_costs[i] = cumulative_costs[i-1] + transaction_costs[i]
            portfolio_values[i] = stock_positions[i] * S + cash_positions[i]
        
        # Calculate hedge errors (difference between hedge portfolio and option liability)
        if self.position == 'short':
            hedge_errors = portfolio_values + option_values  # Should be close to initial premium
        else:
            hedge_errors = portfolio_values - option_values
        
        # Calculate P&L
        final_S = prices[-1]
        if self.is_call:
            payoff = max(final_S - self.K, 0)
        else:
            payoff = max(self.K - final_S, 0)
        
        if self.position == 'short':
            # We owe the payoff
            pnl = portfolio_values - payoff
        else:
            # We receive the payoff
            pnl = portfolio_values + payoff
        
        return HedgingResult(
            times=times,
            prices=prices,
            deltas=deltas,
            option_values=option_values,
            stock_positions=stock_positions,
            cash_positions=cash_positions,
            portfolio_values=portfolio_values,
            hedge_errors=hedge_errors,
            transaction_costs=transaction_costs,
            cumulative_costs=cumulative_costs,
            pnl=pnl
        )

In [ ]:
# Single path delta hedging example
np.random.seed(123)
simulator = GBMSimulator(S0=100, mu=0.05, sigma=0.2)
times, prices = simulator.simulate_path(T=0.25, n_steps=63)  # Daily for 3 months
prices = prices.flatten() if prices.ndim > 1 else prices

# Create hedger for short call option
hedger = DeltaHedger(K=100, T=0.25, r=0.05, sigma=0.2, is_call=True, position='short')

# Run hedging simulation with daily rebalancing
result = hedger.simulate_hedging(prices, times, rebalance_freq=1, transaction_cost_rate=0.0)

# Visualize results
fig, axes = plt.subplots(3, 2, figsize=(14, 12))

# Stock price path
ax1 = axes[0, 0]
ax1.plot(times * 252, result.prices, 'b-', linewidth=1.5)
ax1.axhline(y=100, color='red', linestyle='--', label='Strike')
ax1.set_xlabel('Trading Days')
ax1.set_ylabel('Stock Price ($)')
ax1.set_title('Stock Price Path')
ax1.legend()

# Delta over time
ax2 = axes[0, 1]
ax2.plot(times * 252, result.deltas, 'g-', linewidth=1.5)
ax2.set_xlabel('Trading Days')
ax2.set_ylabel('Delta')
ax2.set_title('Hedge Ratio (Delta) Over Time')
ax2.set_ylim(-0.05, 1.05)

# Option value
ax3 = axes[1, 0]
ax3.plot(times * 252, result.option_values, 'r-', linewidth=1.5)
ax3.set_xlabel('Trading Days')
ax3.set_ylabel('Option Value ($)')
ax3.set_title('Option Value Over Time')

# Portfolio value
ax4 = axes[1, 1]
ax4.plot(times * 252, result.portfolio_values, 'purple', linewidth=1.5, label='Hedge Portfolio')
ax4.plot(times * 252, -result.option_values, 'r--', linewidth=1.5, alpha=0.7, label='-Option Value')
ax4.set_xlabel('Trading Days')
ax4.set_ylabel('Value ($)')
ax4.set_title('Hedge Portfolio vs Option Liability')
ax4.legend()

# Stock position
ax5 = axes[2, 0]
ax5.plot(times * 252, result.stock_positions, 'orange', linewidth=1.5)
ax5.set_xlabel('Trading Days')
ax5.set_ylabel('Shares')
ax5.set_title('Stock Position (Shares Held)')

# Hedge error
ax6 = axes[2, 1]
ax6.plot(times * 252, result.hedge_errors, 'brown', linewidth=1.5)
ax6.axhline(y=result.hedge_errors[0], color='gray', linestyle='--', alpha=0.5)
ax6.set_xlabel('Trading Days')
ax6.set_ylabel('Hedge Error ($)')
ax6.set_title('Cumulative Hedge Error')

plt.tight_layout()
plt.show()

# Print summary
final_payoff = max(result.prices[-1] - 100, 0)  # Call payoff
print("\nHedging Summary")
print("=" * 50)
print(f"Initial Stock Price: ${result.prices[0]:.2f}")
print(f"Final Stock Price: ${result.prices[-1]:.2f}")
print(f"Strike: $100")
print(f"\nInitial Option Premium Received: ${result.option_values[0]:.4f}")
print(f"Final Option Payoff Owed: ${final_payoff:.4f}")
print(f"\nFinal Hedge Portfolio Value: ${result.portfolio_values[-1]:.4f}")
print(f"Net P&L: ${result.pnl[-1]:.4f}")
print(f"\nHedge Error: ${result.hedge_errors[-1] - result.hedge_errors[0]:.4f}")

## 3. Transaction Costs Impact

Transaction costs significantly affect hedging performance. Let's analyze:
- Different rebalancing frequencies
- Impact of proportional costs
- Trade-off between hedge accuracy and costs

In [ ]:
def analyze_rebalancing_frequency(prices: np.ndarray, times: np.ndarray,
                                   frequencies: List[int],
                                   transaction_cost_rate: float = 0.001) -> pd.DataFrame:
    """Analyze hedging performance for different rebalancing frequencies."""
    results = []
    
    hedger = DeltaHedger(K=100, T=0.25, r=0.05, sigma=0.2, is_call=True, position='short')
    
    for freq in frequencies:
        result = hedger.simulate_hedging(prices, times, 
                                         rebalance_freq=freq,
                                         transaction_cost_rate=transaction_cost_rate)
        
        n_rebalances = len(times) // freq
        final_error = result.hedge_errors[-1] - result.hedge_errors[0]
        total_costs = result.cumulative_costs[-1]
        
        results.append({
            'Frequency': f'Every {freq} day(s)',
            'N_Rebalances': n_rebalances,
            'Total_Costs': total_costs,
            'Hedge_Error': final_error,
            'Final_PnL': result.pnl[-1]
        })
    
    return pd.DataFrame(results)


# Analyze different frequencies
np.random.seed(42)
simulator = GBMSimulator(S0=100, mu=0.05, sigma=0.2)
times, prices = simulator.simulate_path(T=0.25, n_steps=63)
prices = prices.flatten() if prices.ndim > 1 else prices

frequencies = [1, 2, 5, 10, 21]  # Daily, every 2 days, weekly, bi-weekly, monthly

# Without transaction costs
df_no_costs = analyze_rebalancing_frequency(prices, times, frequencies, 0.0)
df_no_costs['Scenario'] = 'No Costs'

# With 10bps transaction costs
df_with_costs = analyze_rebalancing_frequency(prices, times, frequencies, 0.001)
df_with_costs['Scenario'] = '10bps Costs'

# With 50bps transaction costs
df_high_costs = analyze_rebalancing_frequency(prices, times, frequencies, 0.005)
df_high_costs['Scenario'] = '50bps Costs'

print("Hedging Performance by Rebalancing Frequency")
print("=" * 70)
print("\nNo Transaction Costs:")
print(df_no_costs[['Frequency', 'N_Rebalances', 'Hedge_Error', 'Final_PnL']].to_string(index=False))
print("\nWith 10bps Transaction Costs:")
print(df_with_costs[['Frequency', 'N_Rebalances', 'Total_Costs', 'Hedge_Error', 'Final_PnL']].to_string(index=False))
print("\nWith 50bps Transaction Costs:")
print(df_high_costs[['Frequency', 'N_Rebalances', 'Total_Costs', 'Hedge_Error', 'Final_PnL']].to_string(index=False))

In [ ]:
# Visualize transaction costs impact
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Compare hedge errors
ax1 = axes[0]
x = range(len(frequencies))
width = 0.25
ax1.bar([i - width for i in x], df_no_costs['Hedge_Error'], width, label='No Costs', alpha=0.8)
ax1.bar(x, df_with_costs['Hedge_Error'], width, label='10bps', alpha=0.8)
ax1.bar([i + width for i in x], df_high_costs['Hedge_Error'], width, label='50bps', alpha=0.8)
ax1.set_xticks(x)
ax1.set_xticklabels([f.split()[1] for f in df_no_costs['Frequency']])
ax1.set_xlabel('Rebalancing Frequency (days)')
ax1.set_ylabel('Hedge Error ($)')
ax1.set_title('Hedge Error by Frequency')
ax1.legend()
ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

# Transaction costs
ax2 = axes[1]
ax2.bar([i - width/2 for i in x], df_with_costs['Total_Costs'], width, label='10bps', alpha=0.8)
ax2.bar([i + width/2 for i in x], df_high_costs['Total_Costs'], width, label='50bps', alpha=0.8)
ax2.set_xticks(x)
ax2.set_xticklabels([f.split()[1] for f in df_no_costs['Frequency']])
ax2.set_xlabel('Rebalancing Frequency (days)')
ax2.set_ylabel('Total Transaction Costs ($)')
ax2.set_title('Cumulative Transaction Costs')
ax2.legend()

# Final P&L
ax3 = axes[2]
ax3.bar([i - width for i in x], df_no_costs['Final_PnL'], width, label='No Costs', alpha=0.8)
ax3.bar(x, df_with_costs['Final_PnL'], width, label='10bps', alpha=0.8)
ax3.bar([i + width for i in x], df_high_costs['Final_PnL'], width, label='50bps', alpha=0.8)
ax3.set_xticks(x)
ax3.set_xticklabels([f.split()[1] for f in df_no_costs['Frequency']])
ax3.set_xlabel('Rebalancing Frequency (days)')
ax3.set_ylabel('Final P&L ($)')
ax3.set_title('Final P&L by Frequency')
ax3.legend()
ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

## 4. P&L Analysis (Monte Carlo)

Let's analyze the distribution of hedging P&L across many simulated paths.

In [ ]:
def monte_carlo_hedging_analysis(n_simulations: int = 1000,
                                  S0: float = 100,
                                  K: float = 100,
                                  T: float = 0.25,
                                  r: float = 0.05,
                                  sigma: float = 0.2,
                                  rebalance_freq: int = 1,
                                  transaction_cost_rate: float = 0.0) -> dict:
    """Run Monte Carlo analysis of delta hedging."""
    
    n_steps = int(T * 252)  # Daily steps
    
    final_pnls = []
    hedge_errors = []
    total_costs = []
    final_prices = []
    
    hedger = DeltaHedger(K=K, T=T, r=r, sigma=sigma, is_call=True, position='short')
    
    for i in range(n_simulations):
        # Simulate price path
        simulator = GBMSimulator(S0=S0, mu=r, sigma=sigma)  # Risk-neutral
        times, prices = simulator.simulate_path(T=T, n_steps=n_steps)
        prices = prices.flatten() if prices.ndim > 1 else prices
        
        # Run hedging
        result = hedger.simulate_hedging(prices, times, 
                                         rebalance_freq=rebalance_freq,
                                         transaction_cost_rate=transaction_cost_rate)
        
        final_pnls.append(result.pnl[-1])
        hedge_errors.append(result.hedge_errors[-1] - result.hedge_errors[0])
        total_costs.append(result.cumulative_costs[-1])
        final_prices.append(prices[-1])
    
    return {
        'final_pnls': np.array(final_pnls),
        'hedge_errors': np.array(hedge_errors),
        'total_costs': np.array(total_costs),
        'final_prices': np.array(final_prices)
    }


# Run Monte Carlo simulation
print("Running Monte Carlo simulation (1000 paths)...")
mc_results_daily = monte_carlo_hedging_analysis(
    n_simulations=1000, 
    rebalance_freq=1,
    transaction_cost_rate=0.001
)

mc_results_weekly = monte_carlo_hedging_analysis(
    n_simulations=1000, 
    rebalance_freq=5,
    transaction_cost_rate=0.001
)

print("Done!")

In [ ]:
# Analyze results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# P&L Distribution - Daily
ax1 = axes[0, 0]
ax1.hist(mc_results_daily['final_pnls'], bins=50, alpha=0.7, edgecolor='black', density=True)
ax1.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax1.axvline(x=np.mean(mc_results_daily['final_pnls']), color='green', linestyle='-', linewidth=2, label=f'Mean: {np.mean(mc_results_daily["final_pnls"]):.3f}')
ax1.set_xlabel('Final P&L ($)')
ax1.set_ylabel('Density')
ax1.set_title('P&L Distribution (Daily Rebalancing, 10bps costs)')
ax1.legend()

# P&L Distribution - Weekly
ax2 = axes[0, 1]
ax2.hist(mc_results_weekly['final_pnls'], bins=50, alpha=0.7, edgecolor='black', density=True, color='orange')
ax2.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax2.axvline(x=np.mean(mc_results_weekly['final_pnls']), color='green', linestyle='-', linewidth=2, label=f'Mean: {np.mean(mc_results_weekly["final_pnls"]):.3f}')
ax2.set_xlabel('Final P&L ($)')
ax2.set_ylabel('Density')
ax2.set_title('P&L Distribution (Weekly Rebalancing, 10bps costs)')
ax2.legend()

# Hedge Error vs Final Price
ax3 = axes[1, 0]
ax3.scatter(mc_results_daily['final_prices'], mc_results_daily['hedge_errors'], alpha=0.3, s=10)
ax3.axhline(y=0, color='red', linestyle='--')
ax3.axvline(x=100, color='gray', linestyle='--', alpha=0.5)
ax3.set_xlabel('Final Stock Price ($)')
ax3.set_ylabel('Hedge Error ($)')
ax3.set_title('Hedge Error vs Final Stock Price (Daily)')

# Comparison box plot
ax4 = axes[1, 1]
data_to_plot = [mc_results_daily['final_pnls'], mc_results_weekly['final_pnls']]
bp = ax4.boxplot(data_to_plot, labels=['Daily', 'Weekly'], patch_artist=True)
colors = ['lightblue', 'lightorange']
for patch, color in zip(bp['boxes'], ['lightblue', 'moccasin']):
    patch.set_facecolor(color)
ax4.axhline(y=0, color='red', linestyle='--')
ax4.set_ylabel('Final P&L ($)')
ax4.set_title('P&L Comparison: Daily vs Weekly Rebalancing')

plt.tight_layout()
plt.show()

# Summary statistics
print("\nMonte Carlo Results Summary (1000 simulations)")
print("=" * 60)
print("\nDaily Rebalancing (10bps transaction costs):")
print(f"  Mean P&L: ${np.mean(mc_results_daily['final_pnls']):.4f}")
print(f"  Std P&L: ${np.std(mc_results_daily['final_pnls']):.4f}")
print(f"  5th percentile: ${np.percentile(mc_results_daily['final_pnls'], 5):.4f}")
print(f"  95th percentile: ${np.percentile(mc_results_daily['final_pnls'], 95):.4f}")
print(f"  Mean transaction costs: ${np.mean(mc_results_daily['total_costs']):.4f}")

print("\nWeekly Rebalancing (10bps transaction costs):")
print(f"  Mean P&L: ${np.mean(mc_results_weekly['final_pnls']):.4f}")
print(f"  Std P&L: ${np.std(mc_results_weekly['final_pnls']):.4f}")
print(f"  5th percentile: ${np.percentile(mc_results_weekly['final_pnls'], 5):.4f}")
print(f"  95th percentile: ${np.percentile(mc_results_weekly['final_pnls'], 95):.4f}")
print(f"  Mean transaction costs: ${np.mean(mc_results_weekly['total_costs']):.4f}")

## 5. Key Takeaways

### Delta Hedging Fundamentals
1. **Delta** measures option sensitivity to underlying price changes
2. **Delta hedging** creates a locally risk-neutral portfolio
3. **Gamma** determines how quickly delta changes (rebalancing need)

### Practical Considerations
1. **Discrete Hedging**: In practice, we can't hedge continuously
   - More frequent rebalancing → smaller hedge errors
   - But also higher transaction costs

2. **Transaction Costs**: Create a trade-off
   - Balance hedge accuracy vs. trading costs
   - Optimal frequency depends on cost structure

3. **Gamma Risk**: Near expiration, ATM options have high gamma
   - Delta changes rapidly
   - Requires more frequent rebalancing

### Next Steps: Deep Hedging
- Neural networks can learn optimal hedging strategies
- Can incorporate transaction costs directly in optimization
- May outperform Black-Scholes delta in realistic settings

In [ ]:
# Summary comparison
print("\n" + "="*60)
print("DELTA HEDGING SUMMARY")
print("="*60)
print("\nKey Formulas:")
print("  Delta (call): Δ = N(d₁)")
print("  Delta (put):  Δ = N(d₁) - 1")
print("  Gamma:        Γ = N'(d₁) / (S·σ·√T)")
print("\nHedging Strategy:")
print("  Short call → Long Δ shares")
print("  Short put  → Short |Δ| shares")
print("\nTrade-offs:")
print("  Frequent rebalancing: Lower hedge error, higher costs")
print("  Less frequent: Higher hedge error, lower costs")
print("\nLimitations of BS Delta Hedging:")
print("  - Assumes constant volatility")
print("  - Ignores transaction costs in derivation")
print("  - Requires continuous rebalancing (impossible in practice)")